[Weather Data](https://www.kaggle.com/datasets/prasad22/weather-data)


# Transform ETL


In [15]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
import psycopg2

Create file config.yaml :


```
database:
  url:  "your db url supabase"
  user: "postgres.user"  
  password: "your password"  
  name: "postgres"
  port: 5432
  table: "table name"
```



Example :   postgresql://postgres.[user]:[YOUR-PASSWORD]@[url]:5432/postgres

In [16]:
def load_config(file_path="/content/config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [17]:
config = load_config()
db_config = config["database"]

# Charge credentials
SUPABASE_URL = db_config["url"]
DB_USER = db_config["user"]
DB_PASSWORD = db_config["password"]
DB_NAME = db_config["name"]
DB_PORT = db_config["port"]
TABLE_NAME = db_config["table"]



In [18]:
# Create conection to DB supabase
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{SUPABASE_URL}:{DB_PORT}/{DB_NAME}")

Read DB

In [19]:
# Read data from Supabase
df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", engine)
# Display the first 5 rows
df.tail(5)

,id,Location,Date_Time,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
10995,10996,San Jose,2024-01-21 01:22:18,26.610890,87.424733,7.197925,5.749313
10996,10997,Dallas,2024-03-07 11:37:08,28.178715,86.330654,2.093757,10.659999
10997,10998,San Jose,2024-01-19 01:03:35,38.268248,66.925891,3.060138,17.813180
10998,10999,San Jose,2024-04-23 18:56:54,19.845844,36.820104,3.341647,14.061083
10999,11000,San Diego,2024-02-05 11:08:00,23.833778,81.874137,8.119941,13.543413


In [20]:
df.dtypes

,0
id,int64
Location,object
Date_Time,object
Temperature_C,float64
Humidity_pct,float64
Precipitation_mm,float64
Wind_Speed_kmh,float64


In [21]:
# Convert Date_Time column to datetime format
df["Date_Time"] = pd.to_datetime(df["Date_Time"])

# Extract year, month, day, hour, and minute from Date_Time
df["Year"] = df["Date_Time"].dt.year
df["Month"] = df["Date_Time"].dt.month
df["Day"] = df["Date_Time"].dt.day
df["Hour"] = df["Date_Time"].dt.hour
df["Minute"] = df["Date_Time"].dt.minute

In [22]:
# Convert Location to uppercase
df["Location"] = df["Location"].str.upper()

# Normalize Temperature_C to Fahrenheit
df["Temperature_F"] = (df["Temperature_C"] * 9/5) + 32

# Normalize Wind_Speed_kmh to mph
df["Wind_Speed_mph"] = df["Wind_Speed_kmh"] * 0.621371


In [24]:
# Create another DataFrame for merging with  of average temperature per location
avg_temp_df = df.groupby("Location")["Temperature_C"].mean().reset_index()
avg_temp_df.rename(columns={"Temperature_C": "Avg_Temperature_C"}, inplace=True)

# Merge the original DataFrame with the aggregated DataFrame
df_merged = df.merge(avg_temp_df, on="Location", how="left")

# Display the transformed and merged DataFrame
df_merged.head()

,id,Location,Date_Time,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh,Year,Month,Day,Hour,Minute,Temperature_F,Wind_Speed_mph,Avg_Temperature_C
0,1,SAN DIEGO,2024-01-14 21:12:46,10.683001,41.195754,4.020119,8.233540,2024,1,14,21,12,51.229402,5.116083,14.858508
1,2,SAN DIEGO,2024-05-17 15:22:10,8.734140,58.319107,9.111623,27.715161,2024,5,17,15,22,47.721452,17.221397,14.858508
2,3,SAN DIEGO,2024-05-11 09:30:59,11.632436,38.820175,4.607511,28.732951,2024,5,11,9,30,52.938385,17.853823,14.858508
3,4,PHILADELPHIA,2024-02-26 17:32:39,-8.628976,54.074474,3.183720,26.367303,2024,2,26,17,32,16.467843,16.383877,14.980439
4,5,SAN ANTONIO,2024-04-29 13:23:51,39.808213,72.899908,9.598282,29.898622,2024,4,29,13,23,103.654783,18.578136,14.302554


In [26]:
# Upload transformed data back to Supabase
df_merged.to_sql(TABLE_NAME, engine, if_exists="replace", index=False)

1000

In [28]:
# Read data from Supabase
df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", engine)
# Display the first 5 rows
df.tail(5)

,id,Location,Date_Time,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh,Year,Month,Day,Hour,Minute,Temperature_F,Wind_Speed_mph,Avg_Temperature_C
10995,10996,SAN JOSE,2024-01-21 01:22:18,26.610890,87.424733,7.197925,5.749313,2024,1,21,1,22,79.899602,3.572457,15.470219
10996,10997,DALLAS,2024-03-07 11:37:08,28.178715,86.330654,2.093757,10.659999,2024,3,7,11,37,82.721687,6.623814,14.884943
10997,10998,SAN JOSE,2024-01-19 01:03:35,38.268248,66.925891,3.060138,17.813180,2024,1,19,1,3,100.882847,11.068594,15.470219
10998,10999,SAN JOSE,2024-04-23 18:56:54,19.845844,36.820104,3.341647,14.061083,2024,4,23,18,56,67.722519,8.737149,15.470219
10999,11000,SAN DIEGO,2024-02-05 11:08:00,23.833778,81.874137,8.119941,13.543413,2024,2,5,11,8,74.900800,8.415484,14.858508
